#Étape 1 : Configuration de l'environnement

In [ ]:
pip install transformers --upgrade

# Étape 2 : Chargement d'AraBERT

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

In [ ]:
# Charger le tokenizer et le modèle AraBERT
model_name = "aubmindlab/bert-base-arabertv02"  # Modèle AraBERT
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)  # 5 classes pour les villes

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/825k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Exemple d'entrée
text_samples = ["وحشوني السلام عليكم"]

# Tokenization
tokens = tokenizer(text_samples, padding=True, truncation=True, max_length=128, return_tensors="tf")

# Passage dans le modèle
outputs = model(tokens['input_ids'])
print(outputs.logits)

tf.Tensor([[-0.3209527  -0.39042705  0.01636633 -0.2946149   0.37811995]], shape=(1, 5), dtype=float32)


# Étape 3 : Préparation des données

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data_casa = pd.read_csv('cleaned_transcript_casablanca.csv')
data_marrakesh = pd.read_csv('cleaned_transcript_marrakesh.csv')
data_tanger = pd.read_csv('cleaned_transcript_tanger.csv')
data_agadir = pd.read_csv('cleaned_transcript_agadir.csv')
data_oujda = pd.read_csv('cleaned_transcript_oujda.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'cleaned_transcript_casablanca.csv'

In [ ]:
print('len(casa) = ',len(data_casa))
print('len(marrakesh) = ',len(data_marrakesh))
print('len(tanger) = ',len(data_tanger))
print('len(agadir) = ',len(data_agadir))
print('len(oujda) = ',len(data_oujda))

len(casa) =  3961
len(marrakesh) =  182486
len(tanger) =  7981
len(agadir) =  61705
len(oujda) =  4468


In [ ]:
# Ajouter une colonne pour indiquer la ville d'origine
data_casa['city'] = 'Casablanca'
data_marrakesh['city'] = 'Marrakesh'
data_tanger['city'] = 'Tanger'
data_agadir['city'] = 'Agadir'
data_oujda['city'] = 'Oujda'

In [ ]:
# Fusionner tous les datasets
df_global = pd.concat([data_casa, data_marrakesh, data_tanger, data_agadir, data_oujda], ignore_index=True)

In [ ]:
# Vérifier la forme du DataFrame global
print("Shape of global dataset:", df_global.shape)  # Devrait être (3961 * 5, n_colonnes)

Shape of global dataset: (260601, 2)


In [ ]:
df_global.head(20)

,text,city
0,وحشوني السلام عليكم,Casablanca
1,الاوراق كاملين ديالهم,Casablanca
2,جديد مرحبا ان شاء الله شي حاجه اخرى,Casablanca
3,ديال الدراري اللي مشاو هذا العام,Casablanca
4,لكم وهذا الشيء اللي كاين شي حاجه اخرى,Casablanca
5,اخر مره,Casablanca
6,قول لهم,Casablanca
7,ولكن فاش غايجي مباشره,Casablanca
8,الحمد لله 100 بعدا اول حاجه السلام,Casablanca
9,عليكم الدراري,Casablanca


In [ ]:
df_global['text'] = df_global['text'].str.replace(r'\d+', '', regex=True)  # Supprimer les chiffres
df_global['text'] = df_global['text'].str.strip()                          # Supprimer les espaces en début/fin
df_global['text'] = df_global['text'].str.replace(r'\s+', ' ', regex=True) # Remplacer plusieurs espaces par un seul

In [ ]:
# Préparation des étiquettes
city_to_id = {city: idx for idx, city in enumerate(df_global['city'].unique())}
df_global['labels'] = df_global['city'].map(city_to_id)

# Tokenisation des textes
X_tokens = tokenizer(df_global['text'].tolist(), padding=True, truncation=True, max_length=128, return_tensors="tf")
y_labels = tf.convert_to_tensor(df_global['labels'].values)

In [ ]:
X_tokens

{'input_ids': <tf.Tensor: shape=(260601, 26), dtype=int32, numpy=
array([[    2, 35217,  1314, ...,     0,     0,     0],
       [    2, 19633, 54838, ...,     0,     0,     0],
       [    2,  1463, 24570, ...,     0,     0,     0],
       ...,
       [    2,  9692, 20261, ...,     0,     0,     0],
       [    2, 12989,   195, ...,     0,     0,     0],
       [    2, 44939,   647, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(260601, 26), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(260601, 26), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype

In [ ]:
y_labels

<tf.Tensor: shape=(260601,), dtype=int64, numpy=array([0, 0, 0, ..., 4, 4, 4])>

# Étape 4 : Entraînement

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [ ]:
!pip list | grep tensorflow
!pip list | grep keras

tensorflow                         2.17.0
tensorflow-datasets                4.9.7
tensorflow-estimator               2.12.0
tensorflow-hub                     0.16.1
tensorflow-io-gcs-filesystem       0.37.1
tensorflow-metadata                1.13.1
tensorflow-probability             0.24.0
keras                              3.7.0
tf_keras                           2.17.0


In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

initial_learning_rate = 1e-4
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,  # Ajustez en fonction de vos données
    decay_rate=0.96,
    staircase=True
)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(
    optimizer='adam',  # Spécifiez l'optimiseur par son nom en chaîne
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
import tensorflow as tf
print("GPU Available: ", tf.config.list_physical_devices('GPU'))

GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[0], 'GPU')

In [ ]:
tf.config.experimental.set_memory_growth(physical_devices[0], True)


RuntimeError: Physical devices cannot be modified after being initialized

In [ ]:
history = model.fit(
    X_tokens['input_ids'],
    y_labels,
    validation_split=0.2,
    epochs=3,  # Réduire temporairement pour des tests rapides
    batch_size=16
)

Epoch 1/3
13030/13030 [==============================] - 1510s 113ms/step - loss: 2.0096 - accuracy: 0.8752 - val_loss: 16.1175 - val_accuracy: 0.0000e+00
Epoch 2/3
 2281/13030 [====>.........................] - ETA: 18:12 - loss: 2.0108 - accuracy: 0.8752

In [ ]:
# 4. Afficher les résultats d'entraînement
print(f"Final training accuracy: {history.history['accuracy'][-1]}")
print(f"Final validation accuracy: {history.history['val_accuracy'][-1]}")

Final training accuracy: 0.9279547333717346
Final validation accuracy: 0.09009803831577301


# Ajustement du Modèle pour Éviter le Surapprentissage

In [ ]:
# Modèle avec Dropout et L2 Regularization
model = TFAutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=5
)

model.classifier = tf.keras.Sequential([
    model.classifier,
    layers.Dropout(0.3),
    layers.Dense(5, activation='softmax', kernel_regularizer=regularizers.l2(0.01))
])

# Planificateur de taux d'apprentissage
initial_learning_rate = 1e-4
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True
)
optimizer = Adam(learning_rate=lr_schedule)

# Compilation du modèle
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Early Stopping pour éviter le surapprentissage
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Entraînement du modèle
history = model.fit(
    X_tokens['input_ids'],
    y_labels,
    validation_split=0.2,
    epochs=10,
    batch_size=16,
    callbacks=[early_stopping]
)

<h1>Étape 1 : Redéfinir la couche de classification avec Dropout

In [ ]:
from tensorflow.keras.layers import Dropout, Dense
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

# Charger le modèle pré-entraîné
model_name = "aubmindlab/bert-base-arabertv02"  # Modèle AraBERT
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTor

In [ ]:
# Extraire la partie de l'encodeur BERT
bert_output = model.bert

# Préparer les entrées
input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name='input_ids')  # Taille d'entrée
attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name='attention_mask')  # Masque d'attention

# Créez des tensors classiques TensorFlow pour les entrées
input_ids_tensor = tf.convert_to_tensor(X_tokens['input_ids'], dtype=tf.int32)
attention_mask_tensor = tf.convert_to_tensor(X_tokens['attention_mask'], dtype=tf.int32)

# Passer les entrées par le modèle BERT
outputs = bert_output(input_ids_tensor, attention_mask=attention_mask_tensor)
pooled_output = outputs.pooler_output  # Récupérer la sortie poolée

# Ajouter des couches supplémentaires
x = Dropout(0.3)(pooled_output)  # Dropout sur la sortie de la dernière couche de BERT (pooler_output)
x = Dense(5, activation='softmax')(x)  # 5 classes pour la classification

# Créer le modèle final
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=x)

# Vérification de la structure du modèle
model.summary()

In [ ]:
# Compiler le modèle
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Entraîner le modèle
history = model.fit(
    {'input_ids': input_ids_tensor, 'attention_mask': attention_mask_tensor},
    y_labels,
    validation_split=0.2,
    epochs=3,
    batch_size=16
)

# Étape 4 : Évaluer le modèle

In [ ]:
# Évaluation sur les données de validation
validation_loss, validation_accuracy = model.evaluate(
    X_tokens['input_ids'],
    y_labels,
    batch_size=16
)

16288/16288 [==============================] - 773s 47ms/step - loss: 1.0175 - accuracy: 0.7841


In [ ]:
print(f"Validation Loss: {validation_loss}")
print(f"Validation Accuracy: {validation_accuracy}")

Validation Loss: 1.017473578453064
Validation Accuracy: 0.7840914130210876


# Étape 5 : Test

In [ ]:
# Exemple de prédiction sur des nouveaux textes
new_texts = ["أنا من الدار البيضاء", "نحب تونس برشا", "القاهرة مليانة ناس"]
new_tokens = tokenizer(new_texts, padding=True, truncation=True, max_length=128, return_tensors="tf")

In [ ]:
# Obtenir les logits et prédictions
logits = model(new_tokens['input_ids']).logits
predictions = tf.argmax(logits, axis=1).numpy()

In [ ]:
# Décoder les prédictions
id_to_city = {idx: city for city, idx in city_to_id.items()}
predicted_cities = [id_to_city[pred] for pred in predictions]

print("Predicted cities for new texts:")
for text, city in zip(new_texts, predicted_cities):
    print(f"Text: {text} --> Predicted City: {city}")

Predicted cities for new texts:
Text: أنا من الدار البيضاء --> Predicted City: Marrakesh
Text: نحب تونس برشا --> Predicted City: Marrakesh
Text: القاهرة مليانة ناس --> Predicted City: Tanger
